
## cn_stock/bars/daily

### Write

command line tools:

```sh
duckdb
.open datas/duckdb/finance.db
drop table bars_daily;
.read finance.duckdb.sql
DESCRIBE bars_daily;
```

or python:

In [11]:
import duckdb

with duckdb.connect("datas/duckdb/finance.db") as conn:
    with open("finance.duckdb.sql") as f:
        conn.execute(f.read())
    conn.table('bars_daily').show()

┌─────────────────────┬──────────┬───────┬───────┬───────┬───┬───────────────┬─────────┬─────────┬─────────┬─────────┐
│         dt          │   name   │ _open │ _high │ _low  │ … │ lb_down_count │  close  │  open   │  high   │   low   │
│     timestamp_s     │ varchar  │ float │ float │ float │   │     uint8     │  float  │  float  │  float  │  float  │
├─────────────────────┼──────────┼───────┼───────┼───────┼───┼───────────────┼─────────┼─────────┼─────────┼─────────┤
│ 1991-01-03 08:00:00 │ 深发展Ａ │  66.4 │  66.4 │  66.4 │ … │             0 │ 7.07077 │ 7.07077 │ 7.07077 │ 7.07077 │
│ 1991-01-04 08:00:00 │ 深发展Ａ │ 66.07 │ 66.07 │ 66.07 │ … │             0 │ 7.03562 │ 7.03562 │ 7.03562 │ 7.03562 │
│ 1991-01-07 08:00:00 │ 深发展Ａ │ 66.42 │ 66.42 │ 66.42 │ … │             0 │  7.0729 │  7.0729 │  7.0729 │  7.0729 │
│ 1991-01-09 08:00:00 │ 深发展Ａ │ 65.76 │ 65.76 │ 65.76 │ … │             0 │ 7.00261 │ 7.00261 │ 7.00261 │ 7.00261 │
│ 1991-01-10 08:00:00 │ 深发展Ａ │ 65.43 │ 65.43 │ 65.43 │ … │      

### Read

In [18]:
import duckdb

def f(conn):
    conn.execute("ATTACH '//DESKTOP-6NI30V4/bt_data/duckdb/finance.db' as finance_db (READ_ONLY)")
    conn.sql("select * from finance_db.bars_daily").fetchnumpy()
    # conn.sql("select dt, name, open from finance_db.bars_daily").fetchnumpy()
    conn.execute("DETACH finance_db")

with duckdb.connect() as conn:
    %timeit f(conn)

2.59 ms ± 223 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
import duckdb
with duckdb.connect() as conn:
    conn.execute("ATTACH 'datas/duckdb/finance.db' as finance_db (READ_ONLY)")
    # df = conn.sql("select * from finance_db.bars_daily").df()
    df = conn.sql("select dt, name, open from finance_db.bars_daily").df()
    conn.execute("DETACH finance_db")
df

,dt,name,open
0,1991-01-03 08:00:00,深发展Ａ,7.07077
1,1991-01-04 08:00:00,深发展Ａ,7.03562
2,1991-01-07 08:00:00,深发展Ａ,7.07290
3,1991-01-09 08:00:00,深发展Ａ,7.00261
4,1991-01-10 08:00:00,深发展Ａ,6.96747
...,...,...,...
7975,2024-08-12 08:00:00,平安银行,10.02000
7976,2024-08-13 08:00:00,平安银行,10.02000
7977,2024-08-14 08:00:00,平安银行,9.94000
7978,2024-08-15 08:00:00,平安银行,9.92000
